## Table and view maintenance
###  Topics
#### 1) Executing an SQL command with no response rows
*  Drop Statistics <BR>
*  Collect Statistics <BR>
#### 2) Executing an SQL command with rows return using cursor processing
NOTE : This technique is intended for small answersets returned and should not be used to bypass the power of the teradataml.DataFrame object <BR>
* Show tables in my datebase created by me and how much space they are using<BR>
* Drop tables<BR>
* Help Statistics - shows statistics for a table<BR>
* Help Table - shows table columns and attributes<BR>
* Help View  - shows view columns<BR>


In [73]:
#  needed to display scrolling data frame outputs below
#!pip install Jinja2

In [74]:
#  imports 
import getpass as gp
import yaml


# for manageing connections
from teradataml import create_context, get_context, remove_context, in_schema

# for setting configure options
from teradataml import configure

# for dropping or viewint tables and views
from teradataml.dbutils.dbutils import db_drop_table, db_drop_view, db_list_tables

# for defining SQL data types
from teradatasqlalchemy.types import *

import pandas as pd
import numpy as np
from IPython.display import display, HTML

# DataFrames
from teradataml.dataframe.dataframe import DataFrame, in_schema
import pandas as pd
import numpy as np

# for dropping tables or views
from teradataml.dbutils.dbutils import db_drop_table, db_drop_view 



In [36]:
#help("teradataml.dbutils.dbutils")

In [37]:
user = 'jsumme17'
#user = gp.getpass("User: ")

In [38]:
password = gp.getpass("Password")

Password ········


In [39]:
host = 'udwprod'
database = 'MLA_BOPS'
temp_database = 'MLA_BOPS'
sourceDB = 'MLA_BOPS'
logmech = 'LDAP'

In [42]:
td_context = create_context(host = host, 
                            username= user, 
                            password = password, 
                            database = database,
                            temp_database_name = database,
                            sslmode = 'ALLOW',
                            logmech=logmech)

/home/jsumme17/conda/lib/python3.8/site-packages/teradataml/context/context.py:480: UserWarning: Warning: Password is URL encoded.
  warnings.warn("Warning: Password is URL encoded.")


### Here we will create a connection, derived from the context object to use with the execute method 

In [43]:
conn = td_context.connect()

#### Here's a simple SQL statement which executes and command and returns no rows (no cursor required) 
This is the Database command, setting the default database for the session.

In [44]:
SQLstr = f"Database {sourceDB};"
print(SQLstr)
conn.execute(SQLstr)

Database MLA_BOPS;


#### next we will create a copy of a table to use for this demo


In [45]:
SQLstr = f"Create table {user}_demo_species as demo_species with data and statistics;"
print(SQLstr)
conn.execute(SQLstr)

Create table jsumme17_demo_species as demo_species with data and statistics;


### Here's set up to create a cursor object, needed for returning result sets.
#### In order to use the sqlalchemy cursor, we have to use the connection.connection object documented in SQLAlchemy
This code does the process in two steps<BR>
  cur_conn = conn.connection<BR>
  cursor_obj = cur_conn.cursor()<BR>
  More information here : https://docs.sqlalchemy.org/en/20/core/connections.html<BR>


In [46]:
# here's a single step, using the original connection context object
cursor_obj = conn.connection.cursor()

#### Run SQL to determine tables and space consumed under a specific database

In [78]:
#  In this example the default database is indentified by {sourceDB}
#  This is also filter on tables only created by the user based on this logic below
#      "  and t.creatorname = user" 
sSelect = f'''Select t.databasename
,t.tablename
,CASE t.Tablekind
WHEN 'T'  then 'Table-indexed'
WHEN 'V'  then 'View'
WHEN 'O'  then 'Table-No Index'
WHEN 'A'  then 'Aggregate function'
WHEN 'P'  then 'Stored Procedure'
WHEN 'I'  then 'Join Index'
WHEN 'X'  then 'Auth. Obj.'
WHEN 'K'  then 'Foreign Server Object'
WHEN '2'  then 'Function Alias Object'
ELSE      'Object type:' || t.Tablekind 
END Object_Type
,sum(coalesce(ts.currentperm,0)) as space_bytes
,max(x.dbperm) as perm_avail
,(space_bytes * 100.000) / perm_avail  as pct_used
from dbc.tablesv t
left join dbc.tablesizev ts
on t.databasename = ts.databasename
and t.tablename = ts.tablename
cross join (sel sum(permspace) as dbperm from dbc.databasesv where databasename = '{sourceDB}' ) x
where t.databasename = '{sourceDB}'
and t.creatorname = user
group by 1,2,3
order by 3,6 desc,1,2;'''
#print(sSelect)

In [50]:
try:
    cursor_obj.execute(sSelect)
    print('Cursor built')
except:
    print('cursor failed')

Cursor built


In [52]:
try:
    dflocal = pd.DataFrame(cursor_obj.fetchall(),columns=[col_desc[0] for col_desc in cursor_obj.description])
    print('Dataframe built')
except:
    print('dataframe build failed')

Dataframe built


In [53]:
# use this to display a table with a scroll bar
display(HTML("<div style='height: 250px; overflow: auto; width: fit-content'>" +
             dflocal.to_html() +
             "</div>"))

,DataBaseName,TableName,Object_Type,space_bytes,perm_avail,pct_used
0,MLA_BOPS,jsumm17_prov_geo_large,Table-indexed,6116876288,3566973212352,0.171
1,MLA_BOPS,jsumm17_members_mn,Table-indexed,1243070464,3566973212352,0.035
2,MLA_BOPS,JSUMME17_sys_queries,Table-indexed,109355008,3566973212352,0.003
3,MLA_BOPS,JSUMME17_sys_queries_mv,Table-indexed,97411072,3566973212352,0.003
4,MLA_BOPS,jsumme17_nicecustaddress20221122,Table-indexed,73908224,3566973212352,0.002
5,MLA_BOPS,jsumm17_prov_geo,Table-indexed,38313984,3566973212352,0.001
6,MLA_BOPS,jsumme17_hier,Table-indexed,19783680,3566973212352,0.001
7,MLA_BOPS,jsumme17_cnty_ab,Table-indexed,7094272,3566973212352,0.000
8,MLA_BOPS,jsumme17_nicecoverageplan20221121,Table-indexed,7077888,3566973212352,0.000
9,MLA_BOPS,jsumme17_svc_accts,Table-indexed,10272768,3566973212352,0.000


### Should I want to drop tables, I can use this code 

In [ ]:
try:
    db_drop_table('jsumme17_demo_species')
    db_drop_table('jsumme17_demo2')
    db_drop_table('jsumme17_demo3')
    db_drop_table('jsumme17_demo4')
except: 
    pass

## Here we will show to display and collect statistics
### First we will issue an SQL command to drop the existing statistics

In [57]:
SQLstr = F'''drop stats on {sourceDB}.jsumme17_demo_species;'''
SQLstr
conn.execute(SQLstr)

### Run the help stats command to show stats on an object
This will return "No Stats Found" because we just dropped the stats on that table

In [59]:
sSelect = f'''help stats {sourceDB}.jsumme17_demo_species;'''
try:
    cursor_obj.execute(sSelect)
    dflocal = pd.DataFrame(cursor_obj.fetchall(),columns=[col_desc[0] for col_desc in cursor_obj.description])
except:
    print('No Stats Found')

No Stats Found


### Now collect statistics on two columns, species_id and genus
##### Options for collecting stats are:
#####  Index version :   Collect stats on mytable  index(indexcol1.... indexcol2);
#####  Column version :   Collect stats on mytable  Column(mycolumn);
    

In [60]:
SQLstr = F'''collect stats on {sourceDB}.jsumme17_demo_species column(species_id);'''
conn.execute(SQLstr)
SQLstr = F'''collect stats on {sourceDB}.jsumme17_demo_species column(genus);'''
conn.execute(SQLstr)

### Now let's see the stats we created 

In [79]:
sSelect = f'''help stats {sourceDB}.jsumme17_demo_species;'''
try:
    cursor_obj.execute(sSelect)
    dflocal = pd.DataFrame(cursor_obj.fetchall(),columns=[col_desc[0] for col_desc in cursor_obj.description])
except:
    print('no stats found')

no stats found


In [63]:
# use this to display a table with a scroll bar
display(HTML("<div style='height: fit-content; overflow: auto; width: fit-content'>" +
             dflocal.to_html() +
             "</div>"))

,Date,Time,Unique Values,Column Names,Column Dictionary Names,Column SQL Names,Column Names UEscape
0,23/03/03,11:57:39,54,*,*,"""*""",None
1,23/03/03,11:57:38,54,species_id,species_id,species_id,None
2,23/03/03,11:57:39,30,genus,genus,genus,None


### HELP TABLE is another useful option

In [64]:
sSelect = f'''help table {sourceDB}.jsumme17_demo_species;'''

In [65]:
try:
    cursor_obj.execute(sSelect)
    dflocal = pd.DataFrame(cursor_obj.fetchall(),columns=[col_desc[0] for col_desc in cursor_obj.description])
except:
    print('no columns found')
    pass

In [66]:
display(HTML("<div style='height: 250px; overflow: auto; width: fit-content'>" +
             dflocal.to_html() +
             "</div>"))

,Column Name,Type,Comment,Nullable,Format,Title,Max Length,Decimal Total Digits,Decimal Fractional Digits,Range Low,Range High,UpperCase,Table/View?,Default value,Char Type,IdCol Type,UDT Name,Temporal,Column Dictionary Name,Column SQL Name,Column Name UEscape,Dictionary Title,SQL Title,Title UEscape,UDT Dictionary Name,UDT SQL Name,UDT Name UEscape,Inline Length,Time Series Column Type,Auto Column
0,species_id,CV,None,Y,X(1024),None,2048,None,None,None,None,N,T,None,2,None,None,N,species_id,species_id,,None,None,None,None,None,None,None,None,None
1,genus,CV,None,Y,X(1024),None,2048,None,None,None,None,N,T,None,2,None,None,N,genus,genus,,None,None,None,None,None,None,None,None,None
2,species,CV,None,Y,X(1024),None,2048,None,None,None,None,N,T,None,2,None,None,N,species,species,,None,None,None,None,None,None,None,None,None
3,taxa,CV,None,Y,X(1024),None,2048,None,None,None,None,N,T,None,2,None,None,N,taxa,taxa,,None,None,None,None,None,None,None,None,None


### HELP VIEW shows information about a view

In [75]:
sSelect = '''help view UDWBASESECUREVIEW1.CNTY;'''

In [76]:
try:
    cursor_obj.execute(sSelect)
    dflocal = pd.DataFrame(cursor_obj.fetchall(),columns=[col_desc[0] for col_desc in cursor_obj.description])
except:
    print('no columns found')
    pass

no columns found


In [77]:
pd.set_option('display.max_rows', None)
# use this to display a table with a scroll bar
display(HTML("<div style='height: 450px; overflow: auto; width: fit-content'>" +
             dflocal.to_html() +
             "</div>"))

,Column Name,Type,Comment,Nullable,Format,Title,Max Length,Decimal Total Digits,Decimal Fractional Digits,Range Low,Range High,UpperCase,Table/View?,Default value,Char Type,IdCol Type,UDT Name,Temporal,Column Dictionary Name,Column SQL Name,Column Name UEscape,Dictionary Title,SQL Title,Title UEscape,UDT Dictionary Name,UDT SQL Name,UDT Name UEscape,Inline Length,Time Series Column Type
0,CNTY_CD,None,"County Code (CNTY_CD) uniquely identifies counties and county equivalents in the United States, certain U.S. possessions, and certain freely associated states.",None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,CNTY_CD,CNTY_CD,,None,None,None,None,None,None,None,None
1,CNTY_NM,None,Full Name of the county,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,CNTY_NM,CNTY_NM,,None,None,None,None,None,None,None,None
2,INSRT_BTCH_ID,None,The ETL Batch Id associated with the Insert of the row into the table.,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,INSRT_BTCH_ID,INSRT_BTCH_ID,,None,None,None,None,None,None,None,None
3,INSRT_TS,None,The calendar date the row was inserted into the table.,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,INSRT_TS,INSRT_TS,,None,None,None,None,None,None,None,None
4,LST_UPDT_TS,None,The calendar date the row was last updated in the table.,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,LST_UPDT_TS,LST_UPDT_TS,,None,None,None,None,None,None,None,None
5,LST_UPDT_BTCH_ID,None,The ETL Batch Id associated with the last time the row was changed in the table.,None,None,None,None,None,None,None,None,None,V,None,None,None,None,None,LST_UPDT_BTCH_ID,LST_UPDT_BTCH_ID,,None,None,None,None,None,None,None,None


### Now let's drop the table we created for this demo

In [70]:
try:
    db_drop_table('jsumme17_demo_species')
except: 
    pass

#### Cleanup / remove connection

In [72]:
remove_context()

True